# Reproduction of SQL queries using Pandas 


# Abstract

Understand use and interpretation of Structured Query Language (SQL) queries and reproducing their outputs using Python and the Pandas package. A variety of datasets were used, containing 336,776 flights that departed in 2013 from three New York airports (EWR, JFK and LGA). The datasets used included flight information, two letter carrier codes, airport data, plane data and hourly meteorological data for the three airports. 



Dataset obtained from https://nycflights13.tidyverse.org/index.html

Create a connection object and create the .db file.

Create a table within the database using the DataFrame with the `.to_sql()` method.

Now that the data has been exported, we can issue SQL queries with `pd.read_sql_query()`.


In [1]:
import sqlite3
import numpy as np
import pandas as pd
pd.set_option("display.notebook_repr_html", False)  # disable "rich" output

formats = "ipynb,py:percent"

# Read csv files as a dataframe
flights = pd.read_csv("flights.csv", comment="#")
airlines = pd.read_csv("airlines.csv", comment="#")
airports = pd.read_csv("airports.csv", comment="#")
planes = pd.read_csv("planes.csv", comment="#")
weather = pd.read_csv("weather.csv", comment="#")

In [2]:
flights.head(5)

   year  month  day  dep_time  sched_dep_time  dep_delay  arr_time  \
0  2013      1    1     517.0             515        2.0     830.0   
1  2013      1    1     533.0             529        4.0     850.0   
2  2013      1    1     542.0             540        2.0     923.0   
3  2013      1    1     544.0             545       -1.0    1004.0   
4  2013      1    1     554.0             600       -6.0     812.0   

   sched_arr_time  arr_delay carrier  flight tailnum origin dest  air_time  \
0             819       11.0      UA    1545  N14228    EWR  IAH     227.0   
1             830       20.0      UA    1714  N24211    LGA  IAH     227.0   
2             850       33.0      AA    1141  N619AA    JFK  MIA     160.0   
3            1022      -18.0      B6     725  N804JB    JFK  BQN     183.0   
4             837      -25.0      DL     461  N668DN    LGA  ATL     116.0   

   distance  hour  minute            time_hour  
0      1400     5      15  2013-01-01 05:00:00  
1      1416 

In [3]:
import tempfile, os.path
dbfile = os.path.join(tempfile.mkdtemp(), "sql_and_pandas.db")
print(dbfile)
conn = sqlite3.connect(dbfile)

C:\Users\User\AppData\Local\Temp\tmpkt7dmn_p\sql_and_pandas.db


In [4]:
flights.to_sql("flights", conn, index=False)
airlines.to_sql("airlines", conn, index=False)
airports.to_sql("airport", conn, index=False)
planes.to_sql("planes", conn, index=False)
weather.to_sql("weather", conn, index=False)

26130

## Query 1
The SQL query returns all unique values from the 'engine' column in the 'planes' table, removing duplicates.

In [5]:
# Query 1
query_1_sql = pd.read_sql_query("""
    SELECT DISTINCT engine FROM planes
""", conn)

query_1_p = planes.loc[:, ["engine"]].drop_duplicates().reset_index(drop=True)

pd.testing.assert_frame_equal(query_1_sql, query_1_p)

In Pandas we filter out all rows from the 'engine' column of the 'planes' DataFrame and use the .drop_duplicates() method to obtain all unique values.


## Query 2 
This SQL query returns all unique values from the 'engine' and 'type' columns in the 'planes' table, removing duplicates.

In [6]:
# Query 2
query_2_sql = pd.read_sql_query("""
    SELECT DISTINCT type, engine FROM planes
""", conn)

query_2_p = planes.loc[:, ["type", "engine"]].drop_duplicates().reset_index(drop=True)

pd.testing.assert_frame_equal(query_2_sql, query_2_p)

In Pandas we filtered out all rows from the 'engine' and 'type' columns of the 'planes' DataFrame and use the .drop_duplicates() method to obtain all unique values.

## Query 3

This SQL query selects the 'engine' column from the 'planes' table and 'COUNT(\*)' is used to count the number of rows in each group. The 'GROUP BY' clause groups the rows based on distinct values in the 'engine' column.

In [7]:
# Query 3
query_3_sql = pd.read_sql_query("""
    SELECT COUNT(*), engine FROM planes GROUP BY engine
""", conn)

# In Pandas
query_3_p = planes.groupby("engine", as_index=False).size()
# Re-name columns
query_3_p.columns= ["engine", "COUNT(*)"]
# Re-order columns
query_3_p = query_3_p[["COUNT(*)", "engine"]]

pd.testing.assert_frame_equal(query_3_sql, query_3_p)

In Pandas we use .groupby().size() on the planes DataFrame to get a count of each distinct type of engine

We then need to re-name and re-order the columns to match the SQL query.

## Query 4

This SQL query creates a table containing each unique combination of 'engine' and 'type' from the 'planes table' and includes a count of the number of occurences for each combination. 

In [8]:
# Query 4
query_4_sql = pd.read_sql_query("""
    SELECT COUNT(*), engine, type FROM planes
GROUP BY engine, type
""", conn)

# In Pandas
query_4_p = planes.groupby(["engine", "type"], as_index=False).size()
# Re-name columns
query_4_p.columns= ["engine", "type", "COUNT(*)"]
# Re-order columns
query_4_p = query_4_p[["COUNT(*)", "engine", "type"]]

pd.testing.assert_frame_equal(query_4_sql, query_4_p)

In Pandas we use group the DataFrame by 'engine' and 'type' to get all unique combinations and use .size() to obtain a count of their occurences.
Again we have to re-name and re-order the columns to match the SQL output.

## Query 5
This SQL query calculates the minimum, average and maximum values of the 'year' column' for each unique combination of 'engine' and 'manufacturer' in the 'planes' table.


In [9]:
# Query 5
query_5_sql = pd.read_sql_query("""
    SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer
    FROM planes
    GROUP BY engine, manufacturer
""", conn)

# In Pandas
query_5_p = planes.loc[:, ["engine", "manufacturer", "year"]].groupby(["engine", "manufacturer"])["year"].aggregate([np.min, np.mean, np.max]).reset_index()

# Re-name columns
query_5_p.columns = ["engine", "manufacturer", "MIN(year)", "AVG(year)", "MAX(year)"]

# Re-order columns
query_5_p = query_5_p[["MIN(year)", "AVG(year)", "MAX(year)", "engine", "manufacturer"]]

pd.testing.assert_frame_equal(query_5_sql, query_5_p)

In Pandas we filter out the 'engine', 'manufacturer' and 'year' columns from the 'planes' DataFrame and .groupby() 'engine' and 'manufacturer' to get all unique combinations of these columns. 

We can then use .aggregate() to obtain the minimum, mean and maxmimum of the year for each unique combination.

## Query 6
This SQL query returns all columns and rows in the 'planes' table where the 'speed' column has a non-NULL value.


In [10]:
# Query 6
query_6_sql = pd.read_sql_query("""
    SELECT * FROM planes WHERE speed IS NOT NULL
""", conn)

# In Pandas
query_6_p = planes.loc[~planes.speed.isna(), :].reset_index(drop=True)

pd.testing.assert_frame_equal(query_6_sql, query_6_p)

In Pandas we filter out all rows from all columns where speed is not (~), NULL (.isna()) 

## Query 7
This SQL query returns the 'tailnum' values from the 'planes' table for aircraft that had the seating capacity between 150 and 210 inclusive and where the year was greater than or equal to 2011.


In [11]:
# Query 7
query_7_sql = pd.read_sql_query("""
    SELECT tailnum FROM planes
    WHERE seats BETWEEN 150 AND 210 AND year >= 2011
""", conn)

# In Pandas
query_7_p = planes.loc[(planes.seats >= 150) & (planes.seats <= 210) & (planes.year >= 2011), ["tailnum"]].reset_index(drop=True)

pd.testing.assert_frame_equal(query_7_sql, query_7_p)


In Pandas we filter out rows where: seats >=150 and seats <= 210 and year >= 2011 from the 'tailnum' column of the 'planes' DataFrame.

## Query 8

This SQL query returns the 'tailnum', 'manufacturer' and 'seats' columns from the 'planes' table where the manufacturer was Boeing, Airbus or Embraer and had more than 390 seats.

In [12]:
# Query 8
query_8_sql = pd.read_sql_query("""
    SELECT tailnum, manufacturer, seats FROM planes
    WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390
""", conn)

# In Pandas
query_8_p = (
    planes.loc[(planes.manufacturer.isin(['BOEING', 'AIRBUS', 'EMBRAER'])) & 
               (planes.seats > 390), 
               ["tailnum", "manufacturer", "seats"]].
    reset_index(drop=True)
)

pd.testing.assert_frame_equal(query_8_sql, query_8_p)

In Pandas we filter out rows that contain 'BOEING', 'AIRBUS' and 'EMBRAER' and had more than 390 seats from the 'tailnum', 'manufacturer' and 'seats' columns of the 'planes' DataFrame.

## Query 9

This SQL query returns distinct combinations of 'year' and 'seats' from the 'planes' table for planes with the year greater than or equal to 2012. 
This query is also sorted by ascending year and descending seats.


In [13]:
# Query 9
query_9_sql = pd.read_sql_query("""
    SELECT DISTINCT year, seats FROM planes
    WHERE year >= 2012 ORDER BY year ASC, seats DESC
""", conn)

# In Pandas
query_9_p = (
    planes.loc[(planes.year >= 2012), 
              ["year", "seats"]].sort_values(["year", "seats"], ascending=[True, False])
    .drop_duplicates().reset_index(drop=True)
)

pd.testing.assert_frame_equal(query_9_sql, query_9_p)

In Pandas we filter out rows that have 'year' greater than or equal to 2012 from the 'year' and 'seat' column of the 'planes' table.
We can then .sort_values() by 'year' and 'seats' by ascending and descending respectively and remove duplicates with .drop_duplicates.

## Query 10
This SQL query returns all distinct combinations of 'year' and 'seats' from the 'planes' table for which aircraft had the year 2012 or higher. 
The results are ordered by 'seats' in descending order and 'year' in ascending order.

In [14]:
# Query 10
query_10_sql = pd.read_sql_query("""
    SELECT DISTINCT year, seats FROM planes
    WHERE year >= 2012 ORDER BY seats DESC, year ASC
""", conn)

# In Pandas
query_10_p = (
    planes.loc[(planes.year >= 2012), 
              ["year", "seats"]].sort_values(["seats", "year"], ascending=[False, True])
    .drop_duplicates().reset_index(drop=True)
)

pd.testing.assert_frame_equal(query_10_sql, query_10_p)

In Pandas we filter out rows that have 'year' greater than or equal to 2012 from the 'year' and 'seat' column of the 'planes' table.
We can then .sort_values() by 'seats' and 'year' by descending and ascending respectively and remove duplicates with .drop_duplicates.

## Query 11
This SQL query counts the number of planes with more than 200 seats for each unique 'manufacturer' from the 'planes' table. 


In [15]:
# Query 11
query_11_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) FROM planes
    WHERE seats > 200 GROUP BY manufacturer
""", conn)

# In Pandas
query_11_p = planes.loc[(planes.seats > 200), :].groupby("manufacturer", as_index=False).size()
# Re-name columns
query_11_p.columns = ["manufacturer", "COUNT(*)"]

pd.testing.assert_frame_equal(query_11_sql, query_11_p)

In Pandas we filter out all rows with seats greater than 200 and use .groupby().size() manufacturer to get a count.
We also re-name the columns to match the SQL query.

## Query 12
This SQL query counts the number of planes for each unique manufacturer in the 'planes' table and only returns manufacturers that have counts greater than 10.

In [16]:
# Query 12
query_12_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) FROM planes
    GROUP BY manufacturer HAVING COUNT(*) > 10
""", conn)

# In Pandas
query_12_p = planes.groupby("manufacturer", as_index=False).size()
# Re-name size as it is the same name the size function, avoid using COUNT(*) as it also has poor interactions
query_12_p.columns = ["manufacturer", "count_placeholder"]
# Filter for count_placeholder > 10
query_12_p = query_12_p.loc[(query_12_p.count_placeholder > 10), :].reset_index(drop=True)
# Re-name columns
query_12_p.columns = ["manufacturer", "COUNT(*)"]

pd.testing.assert_frame_equal(query_12_sql, query_12_p)

In Pandas we groupby manufacturer to get unique manufacturers and size() to get a count.
We then filter by rows greater than 10 and rename the columns to match the original query.

## Query 13
This SQL query counts the number of planes for each unique manufacturer in the 'planes' table and only returns manufacturers that have seats greater than 200 and also for which manufacturers have a count greater than 10.

In [17]:
# Query 13
query_13_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) FROM planes
    WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10
""", conn)

# In Pandas
query_13_p = planes.loc[(planes.seats > 200), :].groupby("manufacturer", as_index=False).size()
# Re-name size as it is the same name the size function, avoid using COUNT(*) as it also has poor interactions
query_13_p.columns = ["manufacturer", "count_placeholder"]
# Filter for count_placeholder > 10
query_13_p = query_13_p.loc[(query_13_p.count_placeholder > 10), :].reset_index(drop=True)
# Re-name columns
query_13_p.columns = ["manufacturer", "COUNT(*)"]

pd.testing.assert_frame_equal(query_13_sql, query_13_p)

In Pandas we filter out rows that have seats greater than 200 and group by 'manufacturer' and use .size() to get a count of 'manufacturer'.
Of this we can then filter again to get counts that are greater than 10 and ren-name the columns to match the original query.

## Query 14
This SQL query counts the number of planes for each unique 'manufactuer' from the 'planes' table. It assigns the alias 'howmany' to the count column.
The top 10 results are ordered by descending count.

In [18]:
# Query 14
query_14_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) AS howmany
    FROM planes
    GROUP BY manufacturer
    ORDER BY howmany DESC LIMIT 10
""", conn)

# In Pandas
query_14_p = planes.groupby("manufacturer", as_index=False).size()
# Rename columns
query_14_p.columns = ["manufacturer", "howmany"]
# Sort by "howmany" and get top 10 results
query_14_p = query_14_p.sort_values("howmany", ascending=False).head(10).reset_index(drop=True)

pd.testing.assert_frame_equal(query_14_sql, query_14_p)

In Pandas we first get the count of unique manufacturer's by using .groupby() and .size().
We can rename the columns to rename the count column to 'howmany' then use .sortvalues().head() to sort by descending 'howmany' and get the top 10 results.

## Query 15
This SQL query retrieves all columns from the 'flights' table as we as the 'year', 'speed' and 'seats' columns from the 'planes' table under the alias 'plane_year', 'plane_speed' and 'plane_seats'. 
The query then performs a left join on the 'flights' table with 'planes' using 'tailnum' as the key.
The left join returns all records from the 'flights' table with matches from the 'planes' table, if there is no match then it will insert NULL's in the columns of the 'planes' table.

In [19]:
# Query 15
query_15_sql = pd.read_sql_query("""
    SELECT
        flights.*,
        planes.year AS plane_year,
        planes.speed AS plane_speed,
        planes.seats AS plane_seats
    FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum
""", conn)

# In Pandas
# Filter planes DataFrame to required columns 
planes_modified = planes.loc[:, ["tailnum", "year", "speed", "seats"]]
# Re-name columns
planes_modified.columns = ["tailnum", "plane_year", "plane_speed", "plane_seats"]
# Merge both DataFrames with a left join
query_15_p = pd.merge(flights, planes_modified, on="tailnum", how='left')

pd.testing.assert_frame_equal(query_15_sql, query_15_p)

In Pandas we filter from the 'planes' DataFrame the columns for 'tailnum', 'year', 'speed' and 'seats' and re-name their columns.
Then we can perform the left join with pd.merge() on 'tailnum'.

## Query 16 
This SQL query retrieves all columns from 'planes' and 'airlines' tables for rows that have a match on 'tailnum' in the 'planes' table and 'carrier' in the 'airlines' table with the distinct combinations found from the 'flights' table.

In [20]:
# Query 16
query_16_sql = pd.read_sql_query("""
    SELECT planes.*, airlines.* FROM
    (SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
    INNER JOIN planes ON cartail.tailnum=planes.tailnum
    INNER JOIN airlines ON cartail.carrier=airlines.carrier
""", conn)

# In Pandas
cartail = flights.loc[:, ["tailnum", "carrier"]].drop_duplicates()
query_16_p = pd.merge(planes, cartail, on="tailnum", how='inner')
query_16_p = pd.merge(query_16_p, airlines, on="carrier", how='inner')

# # Re-order columns
query_16_p = query_16_p[["tailnum", "year", "type", "manufacturer", "model", "engines", "seats", "speed", "engine", "carrier", "name"]]
query_16_p = query_16_p.sort_values(["tailnum", "carrier"]).reset_index(drop=True)

pd.testing.assert_frame_equal(query_16_sql, query_16_p)

In Pandas we create a DataFrame called 'cartail' that contains the 'tailnum' and 'carrier' columns from the 'flights' table. 
Using .drop_duplicates removes all duplicates.
We then perform an inner join on the 'planes' DataFram with the 'cartail' DataFrame using 'tailnum' as the primary key. 
Then another inner join on this new DataFrame with the 'airlines' DataFrame using 'carrier' as the primary key.
We then re-order the columns and sort by 'tailnum' and 'carrier' to match the original query.


## Query 17
This SQL query retrieves creates a table that has flights from the origin 'EWR' from the 'flights' table under the alias 'flights2'.
It also creates another table called 'weather2' that contains the 'year', 'month', 'day' as well as the average temperature and humidity under the alias 'atemp' and 'ahumid' respectively, from the 'weather' table.
It then performs a left join on 'flights2' with 'weather2' using 'year', 'month' and 'day' as the primary keys.


In [21]:
# Query 17
query_17_sql = pd.read_sql_query("""
    SELECT
        flights2.*,
        atemp,
        ahumid
    FROM (
        SELECT * FROM flights WHERE origin='EWR'
    ) AS flights2
    LEFT JOIN (
        SELECT
            year, month, day,
            AVG(temp) AS atemp,
            AVG(humid) AS ahumid
        FROM weather
        WHERE origin='EWR'
        GROUP BY year, month, day
    ) AS weather2
    ON flights2.year=weather2.year
    AND flights2.month=weather2.month
    AND flights2.day=weather2.day
""", conn)

# In Pandas
# Filter flights for origin "EWR"
flights2 = flights.loc[(flights.origin=="EWR"), :]

# Filter weather for origin "EWR" and calculate average for temp and humidity
weather2 = (
    weather.loc[(weather.origin=="EWR"), 
                ["year", "month", "day", "temp", "humid"]]
    .groupby(["year", "month", "day"])
    .agg(atemp=("temp", "mean"), ahumid=("humid", "mean")).reset_index()
)

# Left join flight2 with weather2 
query_17_p = pd.merge(flights2, weather2, on=["year", "month", "day"], how='left')

pd.testing.assert_frame_equal(query_17_sql, query_17_p)

In [22]:
conn.close()

In Pandas we create 'flight2' which contains the filtered results of rows that contain the origin 'EWR' from the 'flights' DataFrame.

We also create 'weather2' which contains the columns 'year', 'month', 'day', 'temp' and 'humid' columns and rows that have origin 'EWR' from the 'weather' DataFrame.

We then groupby 'year', 'month' and 'day' and calculate the average temperature and humidity under the column names 'atemp' and 'ahumid'.

Then we can do left join on 'flights2' with 'weather2'.

Lastly we can close the connection to the database since there are no more remaining tasks.


## Conclusion
From large datasets we are able to obtain meaningful data by filtering out data with desired conditions. This can be performed through SQL which generally can be more readable to humans but also through Pandas. 

We can also join different datasets as long as there is a column that we can use as a primary key that appears in both datasets.

Joining two datasets can make the data more messy but also can add additional information to our pre-existing data, this all depends on how well we can work with the new combined data.

Understanding how to obtain the same query through Pandas allows for deeper appreciation and understanding of the Pandas package.

